# Radar

## Introduction
- Discuss what Radar is
- Introduce waves/wavelengths/reflection principles
- Doppler effect?

## Radar Sytem components

### Discussion on dB
- Overview of decibals and why we use them
### Signal Generators
- Brief overview on waveform generation/types of waveforms
### Filters
- Brief overview on filters
### Mixers
- Brief overview on mixers
### Amplifiers
- Brief overview on amplifiers
### Antennas
- Brief overview on antennas
### Putting it all together
- Discuss radar system topologies (superheterodyne receivers)
- Frequency planning

## Radar Range equation
- Derive the equation
- Derive signal to noise ratio from equation
- Derive equations for to show constraints (max range, ...)
- Discuss losses in system (noise figure, noise temperature, ...)

## Topic: System Design
- Design a radar system given constraints

## Topic: RCS Calibration
- Show how we calibrate for RCS measurements